<img width="800px" src="../fidle/img/00-Fidle-header-01.svg"></img>

# <!-- TITLE --> [VAE6] - Variational AutoEncoder (VAE) with CelebA (small)
<!-- DESC --> Episode 6 : Variational AutoEncoder (VAE) with CelebA (small res.)
<!-- AUTHOR : Jean-Luc Parouty (CNRS/SIMaP) -->

## Objectives :
 - Build and train a VAE model with a large dataset in **small resolution(>70 GB)**
 - Understanding a more advanced programming model with **data generator**

The [CelebFaces Attributes Dataset (CelebA)](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html) contains about 200,000 images (202599,218,178,3).  

## What we're going to do :

 - Defining a VAE model
 - Build the model
 - Train it
 - Follow the learning process with Tensorboard


## Step 1 - Setup environment
### 1.1 - Python stuff

In [1]:
import tensorflow as tf
import numpy as np
import os,sys
from importlib import reload

import modules.vae
import modules.data_generator

reload(modules.data_generator)
reload(modules.vae)

from modules.vae  import VariationalAutoencoder
from modules.data_generator import DataGenerator

sys.path.append('..')
import fidle.pwk_ns as ooo

place, datasets_dir = ooo.init()

VariationalAutoencoder.about()
DataGenerator.about()


FIDLE 2020 - Practical Work Module
Version              : 0.57 DEV
Run time             : Sunday 13 September 2020, 12:03:57
TensorFlow version   : 2.0.0
Keras version        : 2.2.4-tf
Current place        : Fidle at IDRIS
Datasets dir         : /gpfswork/rech/mlh/commun/datasets
Update keras cache   : Done

FIDLE 2020 - Variational AutoEncoder (VAE)
TensorFlow version   : 2.0.0
VAE version          : 1.28

FIDLE 2020 - DataGenerator
Version              : 0.4.1


### 1.2 - The good place

In [2]:
train_dir    = f'{datasets_dir}/celeba/clusters-s.train'
test_dir     = f'{datasets_dir}/celeba/clusters-s.test'

## Step 2 - DataGenerator and validation data
Ok, everything's perfect, now let's instantiate our generator for the entire dataset.

In [3]:
data_gen = DataGenerator(train_dir, 32, k_size=1)
x_test   = np.load(f'{test_dir}/images-000.npy')

print(f'Data generator : {len(data_gen)} batchs of {data_gen.batch_size} images, or {data_gen.dataset_size} images')
print(f'x_test : {len(x_test)} images')

Data generator : 6250 batchs of 32 images, or 200000 images
x_test : 2599 images


## Step 3 - Get VAE model

In [4]:
tag = f'CelebA.001-S.{os.getenv("SLURM_JOB_ID","unknown")}'

input_shape = (128, 128, 3)
z_dim       = 200
verbose     = 1

encoder= [ {'type':'Conv2D',          'filters':32, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
         ]

decoder= [ {'type':'Conv2DTranspose', 'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2DTranspose', 'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2DTranspose', 'filters':32, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2DTranspose', 'filters':3,  'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'sigmoid'}
         ]

vae = modules.vae.VariationalAutoencoder(input_shape    = input_shape, 
                                         encoder_layers = encoder, 
                                         decoder_layers = decoder,
                                         z_dim          = z_dim, 
                                         verbose        = verbose,
                                         run_tag        = tag)
vae.save(model=None)

Model initialized.
Outputs will be in  : ./run/CelebA.001-S.266068

 ---------- Encoder -------------------------------------------------- 

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 64, 32)   896         encoder_input[0][0]              
__________________________________________________________________________________________________
dropout (Dropout)               (None, 64, 64, 32)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)

## Step 4 - Compile it

In [5]:
optimizer = tf.keras.optimizers.Adam(1e-4)
# optimizer     = 'adam'
r_loss_factor = 10000

vae.compile(optimizer, r_loss_factor)

Compiled.


## Step 5 - Train
For 10 epochs, adam optimizer :  
- Run time at IDRIS : 1299.77 sec. - 0:21:39
- Run time at GRICAD : 2092.77 sec. - 0:34:52

In [6]:
epochs            = 10
initial_epoch     = 0

In [7]:
vae.train(data_generator    = data_gen,
          x_test            = x_test,
          epochs            = epochs,
          initial_epoch     = initial_epoch
         )

Epoch 1/10
6250/6250 [==============================] - 199s 32ms/step - loss: 344.8686 - vae_r_loss: 297.1828 - vae_kl_loss: 47.6856 - val_loss: 236.2374 - val_vae_r_loss: 190.5777 - val_vae_kl_loss: 45.7505
Epoch 2/10
6250/6250 [==============================] - 193s 31ms/step - loss: 240.1704 - vae_r_loss: 187.3267 - vae_kl_loss: 52.8434 - val_loss: 217.2158 - val_vae_r_loss: 166.9653 - val_vae_kl_loss: 50.3354
Epoch 3/10
6250/6250 [==============================] - 190s 30ms/step - loss: 228.0754 - vae_r_loss: 173.3834 - vae_kl_loss: 54.6923 - val_loss: 209.6491 - val_vae_r_loss: 156.9384 - val_vae_kl_loss: 52.7751
Epoch 4/10
6250/6250 [==============================] - 190s 30ms/step - loss: 222.6995 - vae_r_loss: 167.0435 - vae_kl_loss: 55.6562 - val_loss: 206.5208 - val_vae_r_loss: 152.5815 - val_vae_kl_loss: 54.0307
Epoch 5/10
6250/6250 [==============================] - 194s 31ms/step - loss: 219.5012 - vae_r_loss: 163.2975 - vae_kl_loss: 56.2037 - val_loss: 204.2858 - val_vae

---
<img width="80px" src="../fidle/img/00-Fidle-logo-01.svg"></img>